///---
layout: post
title: "Vim"
comments: true
share: true
date: 2020-05-13 14:00:00
description: Vim 사용을 위한 자료 정리
tags: note
toc: false
sitemap :
    changefreq : daily
    priority : 1.0
///---

# Vim

## 명령어

## 설정

### Neovim 설치

Neovim 최신 안정 버전 설치

```sh
sudo add-apt-repository ppa:neovim-ppa/stable
sudo apt-get update
sudo apt-get install neovim
```

vi를 neovim으로 대체

```sh
sudo update-alternatives --install /usr/bin/vi vi /usr/bin/nvim 60
sudo update-alternatives --config vi 
sudo update-alternatives --install /usr/bin/vim vim /usr/bin/nvim 60 
sudo update-alternatives --config vim 
sudo update-alternatives --install /usr/bin/editor editor /usr/bin/nvim 60
sudo update-alternatives --config editor
```

## Neovim init.vim 설정

Neovim의 설정 파일은 ~/.config/nvim/init.vim에 위치한다.  
https://seulcode.tistory.com/272 을 참고한 [init.vim](/assets/data/vim/init.vim)

### ESC 입력시 자동 영어 변환 (VScode)

Vim에서 한글 입력 후에 ESC를 눌려 Normal 모드로 진입하면 어떤 명령어도 동작되지 않는 문제가 발생한다. 명령어가 영어가 아닌 한글로 입력되면서 벌어지는 문제인데 VScode의 Vim Plugin은 이 문제를 해결하기 위한 방안을 제공한다.

#### Windows

설정 -> 언어 -> 영어(미국) 추가한다.
https://github.com/daipeihust/im-select 에서 [im-select](/assets/data/vim/im-select.zip) 를 다운 받는다.  
Vscode의 환경설정에 다음 내용을 추가한다.

```
"vim.autoSwitchInputMethod.enable": true,
"vim.autoSwitchInputMethod.defaultIM": "1033",
"vim.autoSwitchInputMethod.obtainIMCmd": "D:\\bin\\im-select.exe",
"vim.autoSwitchInputMethod.switchIMCmd": "D:\\bin\\im-select.exe {im}"
```

#### Ubuntu

Nimf 사용자일 경우 Vscode 환경설정에 다음 내용을 추가한다.

```
```

### Capslock -> ESC

#### Windows
AutoHotKey를 설치하고 ahk 확장자 파일을 만들어 아래 내용을 추가한다.
```
Capslock::Esc
Esc::Capslock
```

#### Mint

- 설정 -> 키보드 -> 배치 -> 옵션 -> Capslock 동작 변경
- 위의 방안이 동작 안할 시 .xsession 혹은 .xinirc 혹은 .bashrc에 아래 내용 추가

```
setxkbmap -option caps:swapescape
```

- vscode 환경 설정에 아래 내용 추가

```
"keyboard.dispatch": "keyCode"
```